In [1]:
import numpy as np
import math
import itertools
import picos as pic

In [2]:
def K_fun(m,k,n):

    Fun=0

    for q in range(0,m+1):
        Fun+= math.comb(n-k,m-q)*math.comb(k,q)*(-1)**q
    return Fun 

def comb_g(n,k):

    if n>=0 and k>=0:
        value=math.comb(n,k)
    else:
        value=0

    return value

def Beta(i,j,k,t,n):

    beta=0

    for u in range(n+1):

        beta+= comb_g(u,t)*comb_g(n-2*k,u-k)*comb_g(n-k-u,i-u)*comb_g(n-k-u,j-u)*(-1)**(u-t)

    return beta

In [11]:
def SDP(n,K,d,D):
    
    ##### SDP #####

    pic.available_solvers()
    
    Ap = pic.RealVariable("Ap", n+1)  # A column vector with n+1 elements.
    
    sdp = pic.Problem()
    
    sdp.set_objective('find') 
    
    c=1/K**2
                 
    ##### Adding matrix constraints #####
    
    sdp.add_constraint(Ap[0]==c)
    sdp.add_constraint(Ap[n]==c/K)
    
    ##### NPA #####
        
    ### Constructing Beta matrices ###
    
    
    BMatrix=[]
    
    for k in range(0,int(n/2)+1):
        
        Matrix=[[0]*(n-2*k+1) for _ in range(n-2*k+1)]

        for i in range(k,n-k+1):
            for j in range(k,n-k+1):
                for t in range(n+1):
                    
                    l=i+j-2*t
                    
                    if l>n or t>i or t>j:
                        
                        Matrix[i-k][j-k]+=0
                    
                    else:
                        
                        Matrix[i-k][j-k]+=(1/math.comb(n,l))*Ap[l]*Beta(i,j,k,t,n)
                
        BMatrix.append(pic.block(Matrix))

        #sdp.add_constraint(BMatrix[k] >> 0) #### B_j(x)>>0 ####
    
    ##### Adding K-L constrains #####
    
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])== 0 for i in range(d)) 
    
    ### A_j>=0 ###
    
    for m in range(n+1):
        sdp.add_constraint(sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*Ap[j] for j in range(m+1)) >= 0)
    
    ### K*B_j-A_j>=0 ###
    
    
    for m in range(n+1):   
                                    
        sdp.add_constraint(                              
        sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1)) >= 0)

    ##### Adding Shadow constrains #####
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        sdp.add_constraint(sum(K_fun(n-j,i,n)*Ap[i] for i in range(n+1)) >= 0)
    
    sol = sdp.solve(primals=False,solver='mosek',verbosity=False)
    
    if sol.problemStatus == "feasible":
        return True                    
    elif sol.problemStatus == "infeasible":
        return False
    else:
        return sol.problemStatus 

D=2

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],

[8,0,4],[8,1,3],[8,2,3],[8,3,3],[8,4,2],[8,5,2],[8,6,2],[8,7,1],[8,8,1],
[9,0,4],[9,1,3],[9,2,3],[9,3,3],[9,4,2],[9,5,2],[9,6,2],[9,7,1],[9,8,1],[9,9,1],
[10,0,4],[10,1,4],[10,2,4],[10,3,3],[10,4,3],[10,5,2],[10,6,2],[10,7,2],[10,8,2],[10,9,1],[10,10,1]
]

#Codes=[[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1]]

#Codes=[[10, 8, 2]]

#### Existing qubit codes ####

for code in Codes:
    
    n,K,d=code
    
    check=SDP(n,2**K,d,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")



#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    
    check=SDP(n,2**K,d+1,D)
    
    if check != False:
        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))

print("Done")


#### The program prints True if the sdp is feasible and false if it is infeasible #### 

Linear Feasibility Problem
  find an assignment
  for
    2×1 real variable Ap
  subject to
    Ap[0] = 1
    Ap[1] = 1
    Ap[1] - Ap[0] = 0
    Ap[0] ≥ 0
    -Ap[0] + 2·Ap[1] ≥ 0
    Ap[1] - Ap[0] ≥ 0
    -(Ap[1] - Ap[0]) + 2·(Ap[0] - Ap[1]) ≥ 0
    Ap[0] - Ap[1] ≥ 0
    Ap[0] + Ap[1] ≥ 0
Linear Feasibility Problem
  find an assignment
  for
    2×1 real variable Ap
  subject to
    Ap[0] = 1
    Ap[1] = 1
    Ap[1] - Ap[0] = 0
    Ap[0] ≥ 0
    -Ap[0] + 2·Ap[1] ≥ 0
    Ap[1] - Ap[0] ≥ 0
    -(Ap[1] - Ap[0]) + 2·(Ap[0] - Ap[1]) ≥ 0
    Ap[0] - Ap[1] ≥ 0
    Ap[0] + Ap[1] ≥ 0
Linear Feasibility Problem
  find an assignment
  for
    2×1 real variable Ap
  subject to
    Ap[0] = 0.25
    Ap[1] = 0.125
    2·Ap[1] - Ap[0] = 0
    Ap[0] ≥ 0
    -Ap[0] + 2·Ap[1] ≥ 0
    2·Ap[1] - Ap[0] ≥ 0
    -(2·Ap[1] - Ap[0]) + 2·(2·Ap[0] - Ap[1]) ≥ 0
    Ap[0] - Ap[1] ≥ 0
    Ap[0] + Ap[1] ≥ 0
Linear Feasibility Problem
  find an assignment
  for
    2×1 real variable Ap
  subject to
    Ap[0] = 0.25


Linear Feasibility Problem
  find an assignment
  for
    5×1 real variable Ap
  subject to
    Ap[0] = 0.25
    Ap[4] = 0.125
    2·Ap[-(i-4)] - Ap[i] = 0 ∀ i ∈ [0…1]
    Ap[0] ≥ 0
    -4·Ap[0] + 2·Ap[1] ≥ 0
    6·Ap[0] - 6·Ap[1] + 4·Ap[2] ≥ 0
    -4·Ap[0] + 6·Ap[1] - 8·Ap[2] + 8·Ap[3] ≥ 0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] ≥ 0
    2·Ap[4] - Ap[0] ≥ 0
    -4·(2·Ap[4] - Ap[0]) + 2·(2·Ap[3] - Ap[1]) ≥ 0
    6·(2·Ap[4] - Ap[0]) - 6·(2·Ap[3] - Ap[1]) + 4·(2·Ap[2] - Ap[2]) ≥
      0
    -4·(2·Ap[4] - Ap[0]) + 6·(2·Ap[3] - Ap[1]) - 8·(2·Ap[2] - Ap[2]) +
      8·(2·Ap[1] - Ap[3]) ≥ 0
    2·Ap[4] - Ap[0] - 2·(2·Ap[3] - Ap[1]) + 4·(2·Ap[2] - Ap[2]) -
      8·(2·Ap[1] - Ap[3]) + 16·(2·Ap[0] - Ap[4]) ≥ 0
    Ap[0] - Ap[1] + Ap[2] - Ap[3] + Ap[4] ≥ 0
    4·Ap[0] - 2·Ap[1] + 2·Ap[3] - 4·Ap[4] ≥ 0
    6·Ap[0] - 2·Ap[2] + 6·Ap[4] ≥ 0
    4·Ap[0] + 2·Ap[1] - 2·Ap[3] - 4·Ap[4] ≥ 0
    Ap[0] + Ap[1] + Ap[2] + Ap[3] + Ap[4] ≥ 0
Linear Feasibility Problem
  find an assignment
  for
    5×

Linear Feasibility Problem
  find an assignment
  for
    6×1 real variable Ap
  subject to
    Ap[0] = 0.25
    Ap[5] = 0.125
    2·Ap[-(i-5)] - Ap[i] = 0 ∀ i ∈ [0…2]
    Ap[0] ≥ 0
    -5·Ap[0] + 2·Ap[1] ≥ 0
    10·Ap[0] - 8·Ap[1] + 4·Ap[2] ≥ 0
    -10·Ap[0] + 12·Ap[1] - 12·Ap[2] + 8·Ap[3] ≥ 0
    5·Ap[0] - 8·Ap[1] + 12·Ap[2] - 16·Ap[3] + 16·Ap[4] ≥ 0
    -Ap[0] + 2·Ap[1] - 4·Ap[2] + 8·Ap[3] - 16·Ap[4] + 32·Ap[5] ≥ 0
    2·Ap[5] - Ap[0] ≥ 0
    -5·(2·Ap[5] - Ap[0]) + 2·(2·Ap[4] - Ap[1]) ≥ 0
    10·(2·Ap[5] - Ap[0]) - 8·(2·Ap[4] - Ap[1]) + 4·(2·Ap[3] - Ap[2]) ≥
      0
    -10·(2·Ap[5] - Ap[0]) + 12·(2·Ap[4] - Ap[1]) - 12·(2·Ap[3] -
      Ap[2]) + 8·(2·Ap[2] - Ap[3]) ≥ 0
    5·(2·Ap[5] - Ap[0]) - 8·(2·Ap[4] - Ap[1]) + 12·(2·Ap[3] - Ap[2]) -
      16·(2·Ap[2] - Ap[3]) + 16·(2·Ap[1] - Ap[4]) ≥ 0
    -(2·Ap[5] - Ap[0]) + 2·(2·Ap[4] - Ap[1]) - 4·(2·Ap[3] - Ap[2]) +
      8·(2·Ap[2] - Ap[3]) - 16·(2·Ap[1] - Ap[4]) + 32·(2·Ap[0] -
      Ap[5]) ≥ 0
    Ap[0] - Ap[1] + Ap[2] - Ap[3] + Ap[4] - 

Linear Feasibility Problem
  find an assignment
  for
    7×1 real variable Ap
  subject to
    Ap[0] = 0.25
    Ap[6] = 0.125
    2·Ap[-(i-6)] - Ap[i] = 0 ∀ i ∈ [0…2]
    Ap[0] ≥ 0
    -6·Ap[0] + 2·Ap[1] ≥ 0
    15·Ap[0] - 10·Ap[1] + 4·Ap[2] ≥ 0
    -20·Ap[0] + 20·Ap[1] - 16·Ap[2] + 8·Ap[3] ≥ 0
    15·Ap[0] - 20·Ap[1] + 24·Ap[2] - 24·Ap[3] + 16·Ap[4] ≥ 0
    -6·Ap[0] + 10·Ap[1] - 16·Ap[2] + 24·Ap[3] - 32·Ap[4] + 32·Ap[5] ≥
      0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] ≥ 0
    2·Ap[6] - Ap[0] ≥ 0
    -6·(2·Ap[6] - Ap[0]) + 2·(2·Ap[5] - Ap[1]) ≥ 0
    15·(2·Ap[6] - Ap[0]) - 10·(2·Ap[5] - Ap[1]) + 4·(2·Ap[4] - Ap[2])
      ≥ 0
    -20·(2·Ap[6] - Ap[0]) + 20·(2·Ap[5] - Ap[1]) - 16·(2·Ap[4] -
      Ap[2]) + 8·(2·Ap[3] - Ap[3]) ≥ 0
    15·(2·Ap[6] - Ap[0]) - 20·(2·Ap[5] - Ap[1]) + 24·(2·Ap[4] - Ap[2])
      - 24·(2·Ap[3] - Ap[3]) + 16·(2·Ap[2] - Ap[4]) ≥ 0
    -6·(2·Ap[6] - Ap[0]) + 10·(2·Ap[5] - Ap[1]) - 16·(2·Ap[4] - Ap[2])
      + 24·(2·Ap[3] - Ap

Linear Feasibility Problem
  find an assignment
  for
    7×1 real variable Ap
  subject to
    Ap[0] = 0.000976562
    Ap[6] = 3.05176e-05
    32·Ap[6] - Ap[0] = 0
    Ap[0] ≥ 0
    -6·Ap[0] + 2·Ap[1] ≥ 0
    15·Ap[0] - 10·Ap[1] + 4·Ap[2] ≥ 0
    -20·Ap[0] + 20·Ap[1] - 16·Ap[2] + 8·Ap[3] ≥ 0
    15·Ap[0] - 20·Ap[1] + 24·Ap[2] - 24·Ap[3] + 16·Ap[4] ≥ 0
    -6·Ap[0] + 10·Ap[1] - 16·Ap[2] + 24·Ap[3] - 32·Ap[4] + 32·Ap[5] ≥
      0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] ≥ 0
    32·Ap[6] - Ap[0] ≥ 0
    -6·(32·Ap[6] - Ap[0]) + 2·(32·Ap[5] - Ap[1]) ≥ 0
    15·(32·Ap[6] - Ap[0]) - 10·(32·Ap[5] - Ap[1]) + 4·(32·Ap[4] -
      Ap[2]) ≥ 0
    -20·(32·Ap[6] - Ap[0]) + 20·(32·Ap[5] - Ap[1]) - 16·(32·Ap[4] -
      Ap[2]) + 8·(32·Ap[3] - Ap[3]) ≥ 0
    15·(32·Ap[6] - Ap[0]) - 20·(32·Ap[5] - Ap[1]) + 24·(32·Ap[4] -
      Ap[2]) - 24·(32·Ap[3] - Ap[3]) + 16·(32·Ap[2] - Ap[4]) ≥ 0
    -6·(32·Ap[6] - Ap[0]) + 10·(32·Ap[5] - Ap[1]) - 16·(32·Ap[4] -
      Ap[2]) + 2

...

Linear Feasibility Problem
  find an assignment
  for
    11×1 real variable Ap
  subject to
    Ap[0] = 0.000244141
    Ap[10] = 3.8147e-06
    64·Ap[-(i-10)] - Ap[i] = 0 ∀ i ∈ [0…2]
    Ap[0] ≥ 0
    -10·Ap[0] + 2·Ap[1] ≥ 0
    45·Ap[0] - 18·Ap[1] + 4·Ap[2] ≥ 0
    -120·Ap[0] + 72·Ap[1] - 32·Ap[2] + 8·Ap[3] ≥ 0
    210·Ap[0] - 168·Ap[1] + 112·Ap[2] - 56·Ap[3] + 16·Ap[4] ≥ 0
    -252·Ap[0] + 252·Ap[1] - 224·Ap[2] + 168·Ap[3] - 96·Ap[4] +
      32·Ap[5] ≥ 0
    210·Ap[0] - 252·Ap[1] + 280·Ap[2] - 280·Ap[3] + 240·Ap[4] -
      160·Ap[5] + 64·Ap[6] ≥ 0
    -120·Ap[0] + 168·Ap[1] - 224·Ap[2] + 280·Ap[3] - 320·Ap[4] +
      320·Ap[5] - 256·Ap[6] + 128·Ap[7] ≥ 0
    45·Ap[0] - 72·Ap[1] + 112·Ap[2] - 168·Ap[3] + 240·Ap[4] -
      320·Ap[5] + 384·Ap[6] - 384·Ap[7] + 256·Ap[8] ≥ 0
    -10·Ap[0] + 18·Ap[1] - 32·Ap[2] + 56·Ap[3] - 96·Ap[4] + 160·Ap[5]
      - 256·Ap[6] + 384·Ap[7] - 512·Ap[8] + 512·Ap[9] ≥ 0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] - 128·Ap[

Linear Feasibility Problem
  find an assignment
  for
    11×1 real variable Ap
  subject to
    Ap[0] = 6.10352e-05
    Ap[10] = 4.76837e-07
    128·Ap[-(i-10)] - Ap[i] = 0 ∀ i ∈ [0…2]
    Ap[0] ≥ 0
    -10·Ap[0] + 2·Ap[1] ≥ 0
    45·Ap[0] - 18·Ap[1] + 4·Ap[2] ≥ 0
    -120·Ap[0] + 72·Ap[1] - 32·Ap[2] + 8·Ap[3] ≥ 0
    210·Ap[0] - 168·Ap[1] + 112·Ap[2] - 56·Ap[3] + 16·Ap[4] ≥ 0
    -252·Ap[0] + 252·Ap[1] - 224·Ap[2] + 168·Ap[3] - 96·Ap[4] +
      32·Ap[5] ≥ 0
    210·Ap[0] - 252·Ap[1] + 280·Ap[2] - 280·Ap[3] + 240·Ap[4] -
      160·Ap[5] + 64·Ap[6] ≥ 0
    -120·Ap[0] + 168·Ap[1] - 224·Ap[2] + 280·Ap[3] - 320·Ap[4] +
      320·Ap[5] - 256·Ap[6] + 128·Ap[7] ≥ 0
    45·Ap[0] - 72·Ap[1] + 112·Ap[2] - 168·Ap[3] + 240·Ap[4] -
      320·Ap[5] + 384·Ap[6] - 384·Ap[7] + 256·Ap[8] ≥ 0
    -10·Ap[0] + 18·Ap[1] - 32·Ap[2] + 56·Ap[3] - 96·Ap[4] + 160·Ap[5]
      - 256·Ap[6] + 384·Ap[7] - 512·Ap[8] + 512·Ap[9] ≥ 0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] - 128·A

Linear Feasibility Problem
  find an assignment
  for
    11×1 real variable Ap
  subject to
    Ap[0] = 1.52588e-05
    Ap[10] = 5.96046e-08
    256·Ap[-(i-10)] - Ap[i] = 0 ∀ i ∈ [0…2]
    Ap[0] ≥ 0
    -10·Ap[0] + 2·Ap[1] ≥ 0
    45·Ap[0] - 18·Ap[1] + 4·Ap[2] ≥ 0
    -120·Ap[0] + 72·Ap[1] - 32·Ap[2] + 8·Ap[3] ≥ 0
    210·Ap[0] - 168·Ap[1] + 112·Ap[2] - 56·Ap[3] + 16·Ap[4] ≥ 0
    -252·Ap[0] + 252·Ap[1] - 224·Ap[2] + 168·Ap[3] - 96·Ap[4] +
      32·Ap[5] ≥ 0
    210·Ap[0] - 252·Ap[1] + 280·Ap[2] - 280·Ap[3] + 240·Ap[4] -
      160·Ap[5] + 64·Ap[6] ≥ 0
    -120·Ap[0] + 168·Ap[1] - 224·Ap[2] + 280·Ap[3] - 320·Ap[4] +
      320·Ap[5] - 256·Ap[6] + 128·Ap[7] ≥ 0
    45·Ap[0] - 72·Ap[1] + 112·Ap[2] - 168·Ap[3] + 240·Ap[4] -
      320·Ap[5] + 384·Ap[6] - 384·Ap[7] + 256·Ap[8] ≥ 0
    -10·Ap[0] + 18·Ap[1] - 32·Ap[2] + 56·Ap[3] - 96·Ap[4] + 160·Ap[5]
      - 256·Ap[6] + 384·Ap[7] - 512·Ap[8] + 512·Ap[9] ≥ 0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] - 128·A

Linear Feasibility Problem
  find an assignment
  for
    11×1 real variable Ap
  subject to
    Ap[0] = 3.8147e-06
    Ap[10] = 7.45058e-09
    512·Ap[-(i-10)] - Ap[i] = 0 ∀ i ∈ [0…1]
    Ap[0] ≥ 0
    -10·Ap[0] + 2·Ap[1] ≥ 0
    45·Ap[0] - 18·Ap[1] + 4·Ap[2] ≥ 0
    -120·Ap[0] + 72·Ap[1] - 32·Ap[2] + 8·Ap[3] ≥ 0
    210·Ap[0] - 168·Ap[1] + 112·Ap[2] - 56·Ap[3] + 16·Ap[4] ≥ 0
    -252·Ap[0] + 252·Ap[1] - 224·Ap[2] + 168·Ap[3] - 96·Ap[4] +
      32·Ap[5] ≥ 0
    210·Ap[0] - 252·Ap[1] + 280·Ap[2] - 280·Ap[3] + 240·Ap[4] -
      160·Ap[5] + 64·Ap[6] ≥ 0
    -120·Ap[0] + 168·Ap[1] - 224·Ap[2] + 280·Ap[3] - 320·Ap[4] +
      320·Ap[5] - 256·Ap[6] + 128·Ap[7] ≥ 0
    45·Ap[0] - 72·Ap[1] + 112·Ap[2] - 168·Ap[3] + 240·Ap[4] -
      320·Ap[5] + 384·Ap[6] - 384·Ap[7] + 256·Ap[8] ≥ 0
    -10·Ap[0] + 18·Ap[1] - 32·Ap[2] + 56·Ap[3] - 96·Ap[4] + 160·Ap[5]
      - 256·Ap[6] + 384·Ap[7] - 512·Ap[8] + 512·Ap[9] ≥ 0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] - 128·Ap

Linear Feasibility Problem
  find an assignment
  for
    11×1 real variable Ap
  subject to
    Ap[0] = 9.53674e-07
    Ap[10] = 9.31323e-10
    1024·Ap[-(i-10)] - Ap[i] = 0 ∀ i ∈ [0…1]
    Ap[0] ≥ 0
    -10·Ap[0] + 2·Ap[1] ≥ 0
    45·Ap[0] - 18·Ap[1] + 4·Ap[2] ≥ 0
    -120·Ap[0] + 72·Ap[1] - 32·Ap[2] + 8·Ap[3] ≥ 0
    210·Ap[0] - 168·Ap[1] + 112·Ap[2] - 56·Ap[3] + 16·Ap[4] ≥ 0
    -252·Ap[0] + 252·Ap[1] - 224·Ap[2] + 168·Ap[3] - 96·Ap[4] +
      32·Ap[5] ≥ 0
    210·Ap[0] - 252·Ap[1] + 280·Ap[2] - 280·Ap[3] + 240·Ap[4] -
      160·Ap[5] + 64·Ap[6] ≥ 0
    -120·Ap[0] + 168·Ap[1] - 224·Ap[2] + 280·Ap[3] - 320·Ap[4] +
      320·Ap[5] - 256·Ap[6] + 128·Ap[7] ≥ 0
    45·Ap[0] - 72·Ap[1] + 112·Ap[2] - 168·Ap[3] + 240·Ap[4] -
      320·Ap[5] + 384·Ap[6] - 384·Ap[7] + 256·Ap[8] ≥ 0
    -10·Ap[0] + 18·Ap[1] - 32·Ap[2] + 56·Ap[3] - 96·Ap[4] + 160·Ap[5]
      - 256·Ap[6] + 384·Ap[7] - 512·Ap[8] + 512·Ap[9] ≥ 0
    Ap[0] - 2·Ap[1] + 4·Ap[2] - 8·Ap[3] + 16·Ap[4] - 32·Ap[5] +
      64·Ap[6] - 128·

In [4]:
D=2

Codes=[[7,26,2],[9,112,2],[11,460,2],[13,1877,2],[15,7606,2]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[7, 26, 2]=True
[9, 112, 2]=True
[11, 460, 2]=True
[13, 1877, 2]=True
[15, 7606, 2]=True


In [5]:
D=2

Codes=[[7,3,3],[8,9,3],[9,13,3],[11,53,3],[12,89,3],[13,204,3],[14,324,3],[15,580,3]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[7, 3, 3]=True
[8, 9, 3]=True
[9, 13, 3]=True
[11, 53, 3]=True
[12, 89, 3]=True
[13, 204, 3]=True
[14, 324, 3]=True
[15, 580, 3]=True


In [6]:
D=2

Codes=[[7,1,4],[10,5,4],[11,7,4],[12,20,4],[13,40,4],[14,102,4],[15,150,4]]
    
for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[7, 1, 4]=True
[10, 5, 4]=True
[11, 7, 4]=True
[12, 20, 4]=True
[13, 40, 4]=True
[14, 102, 4]=True
[15, 150, 4]=True


In [7]:
D=2

Codes=[[13,3,5],[14,10,5],[15,18,5]]
    
for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[13, 3, 5]=True
[14, 10, 5]=True
[15, 18, 5]=True


In [8]:
Codes=[[14,3,5],[24,0,10]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],2**codes[1],codes[2],D)))

[14, 3, 5]=True
[24, 0, 10]=True
